<a href="https://colab.research.google.com/github/yeshwandhjaganathan/AutoClaimAI-Vehicle-Damage-Detection-and-Claim-Automation/blob/main/AutoClaimAI_%E2%80%93_Vehicle_Damage_Detection_%26_Claim_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision ultralytics tensorflow pillow numpy imagehash shap opencv-python-headless matplotlib scikit-learn

In [ ]:
import torch
import torchvision
import tensorflow as tf
import PIL
import numpy as np
import imagehash
import shap
import cv2
import matplotlib.pyplot as plt
import os
import io
from ultralytics import YOLO

print("torch:", torch.__version__)
print("torchvision:", torchvision.__version__)
print("tensorflow:", tf.__version__)
print("PIL:", PIL.__version__)
print("numpy:", np.__version__)
print("imagehash:", imagehash.__version__)
print("shap:", shap.__version__)
print("opencv:", cv2.__version__)

print("✅ All libraries loaded successfully!")

In [ ]:
print("Loading YOLOv5 model...")
yolo_model = YOLO('yolov5s.pt')
print("✅ YOLOv5s model loaded!")

In [ ]:

from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

NUM_SEVERITY_CLASSES = 3  # Minor, Moderate, Severe

def create_densenet_model(num_classes, input_shape=(224,224,3)):
    base_model = DenseNet121(weights="imagenet", include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

print("Creating DenseNet model...")
densenet_model = create_densenet_model(NUM_SEVERITY_CLASSES)
densenet_model.summary()
print("✅ DenseNet model ready (ImageNet weights).")

In [20]:
from google.colab import files
import os
import cv2

# Upload files (images/videos)
uploaded_files = files.upload()

uploaded_image_paths = []

# Directory to save extracted frames
frames_dir = "video_frames"
os.makedirs(frames_dir, exist_ok=True)

for filename in uploaded_files.keys():
    extension = filename.split(".")[-1].lower()

    if extension in ["jpg", "jpeg", "png"]:
        # Image file: Save as-is
        with open(filename, 'wb') as f:
            f.write(uploaded_files[filename])
        uploaded_image_paths.append(filename)
        print(f"✅ Image uploaded: {filename}")

    elif extension in ["mp4", "avi", "mov", "mkv"]:
        # Video file: Extract frames
        video_path = filename
        with open(video_path, 'wb') as f:
            f.write(uploaded_files[filename])
        print(f"🎥 Video uploaded: {filename}")

        vidcap = cv2.VideoCapture(video_path)
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        frame_interval = int(fps)  # 1 frame per second

        success, image = vidcap.read()
        count = 0
        frame_count = 0

        while success:
            if count % frame_interval == 0:
                frame_filename = os.path.join(frames_dir, f"{filename}_frame_{frame_count}.jpg")
                cv2.imwrite(frame_filename, image)
                uploaded_image_paths.append(frame_filename)
                print(f"🖼️ Extracted frame: {frame_filename}")
                frame_count += 1
            success, image = vidcap.read()
            count += 1

        vidcap.release()

    else:
        print(f"⚠️ Unsupported file type: {filename}")

if uploaded_image_paths:
    print("\n✅ All files processed. List of images/frames to analyze:")
    for path in uploaded_image_paths:
        print(f"- {path}")
else:
    print("⚠️ No valid images or video frames uploaded.")

TypeError: 'NoneType' object is not subscriptable

In [ ]:
detected_images_info = []

for image_path in uploaded_image_paths:
    print("\nProcessing:", image_path)
    results = yolo_model(image_path)
    for r in results:
        im_array = r.plot()
        im_rgb = PIL.Image.fromarray(im_array[...,::-1])
        plt.figure(figsize=(8,6))
        plt.imshow(im_rgb)
        plt.title("YOLOv5 Detection")
        plt.axis("off")
        plt.show()

        detected_objects = []
        for box in r.boxes:
            x1,y1,x2,y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            class_name = yolo_model.names[cls]
            detected_objects.append({
                "box": [x1,y1,x2,y2],
                "confidence": conf,
                "class_name": class_name
            })

        detected_images_info.append({
            "image_path": image_path,
            "detections": detected_objects
        })

In [ ]:
SEVERITY_LABELS = ["Minor", "Moderate", "Severe"]

classified_images_info = []

for img_info in detected_images_info:
    img_path = img_info["image_path"]
    img = PIL.Image.open(img_path).convert("RGB")
    img_resized = img.resize((224,224))
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = densenet_model.predict(img_array)
    idx = np.argmax(pred[0])
    conf = pred[0][idx]

    print(f"\nImage: {os.path.basename(img_path)} - Severity: {SEVERITY_LABELS[idx]} ({conf*100:.2f}%)")

    classified_images_info.append({
        "image_path": img_path,
        "predicted_severity": SEVERITY_LABELS[idx],
        "confidence": conf,
        "predicted_class_idx": idx
    })

In [ ]:
from PIL import Image
import imagehash

stored_claim_hashes = []  # Simulated previous hashes

def check_duplicate(image_path, stored_hashes, threshold=5):
    img = Image.open(image_path).convert("L")
    current_hash = imagehash.average_hash(img)
    for h in stored_hashes:
        if abs(current_hash - h) <= threshold:
            return True, str(h)
    return False, None

fraud_check_results = []

for img_path in uploaded_image_paths:
    is_dup, hash_match = check_duplicate(img_path, stored_claim_hashes)
    print(f"\nImage: {os.path.basename(img_path)} - Duplicate: {'Yes' if is_dup else 'No'}")
    fraud_check_results.append({
        "image_path": img_path,
        "is_duplicate": is_dup,
        "matching_hash": hash_match if hash_match else "None"
    })


Image: images (15).jpeg - Duplicate: No


In [ ]:
print("\n================= FINAL REPORT =================")
for img_info in classified_images_info:
    img_path = img_info["image_path"]
    img_name = os.path.basename(img_path)
    severity = img_info["predicted_severity"]
    conf_pct = img_info["confidence"] * 100

    detection_info = next((d for d in detected_images_info if d["image_path"]==img_path), None)
    fraud_info = next((f for f in fraud_check_results if f["image_path"]==img_path), None)

    print(f"\nImage: {img_name}")
    print("- Damage Detection:")
    if detection_info and detection_info["detections"]:
        classes = [d["class_name"] for d in detection_info["detections"]]
        confidences = [f"{d['confidence']*100:.2f}%" for d in detection_info["detections"]]
        boxes = [d["box"] for d in detection_info["detections"]]
        print(f"    Objects Detected: {', '.join(classes)}")
        print(f"    Confidence: {confidences}")
        print(f"    Bounding Boxes: {boxes}")
    else:
        print("    No objects detected.")

    print("- Severity Classification:")
    print(f"    Predicted Severity: {severity}")
    print(f"    Confidence: {conf_pct:.2f}%")

    print("- Fraud Check:")
    status = "Unique"
    if fraud_info and fraud_info["is_duplicate"]:
        status = f"Duplicate (Matched Hash: {fraud_info['matching_hash']})"
    print(f"    Status: {status}")

print("\n================= END OF REPORT =================")

In [ ]:
import pandas as pd

final_data = []

for img_info in classified_images_info:
    img_path = img_info["image_path"]
    img_name = os.path.basename(img_path)
    severity = img_info["predicted_severity"]
    severity_conf = round(img_info["confidence"] * 100, 2)

    detection_info = next((d for d in detected_images_info if d["image_path"]==img_path), None)
    fraud_info = next((f for f in fraud_check_results if f["image_path"]==img_path), None)

    if detection_info and detection_info["detections"]:
        detected_classes = ', '.join([d["class_name"] for d in detection_info["detections"]])
        detected_confs = ', '.join([f"{d['confidence']*100:.2f}%" for d in detection_info["detections"]])
        detected_boxes = str([d["box"] for d in detection_info["detections"]])
    else:
        detected_classes = "None"
        detected_confs = "None"
        detected_boxes = "None"

    is_dup = "Yes" if fraud_info and fraud_info["is_duplicate"] else "No"
    hash_match = fraud_info["matching_hash"] if fraud_info and fraud_info["is_duplicate"] else "N/A"

    final_data.append({
        "Image Name": img_name,
        "Detected Objects": detected_classes,
        "Detection Confidence": detected_confs,
        "Bounding Boxes": detected_boxes,
        "Predicted Severity": severity,
        "Severity Confidence (%)": severity_conf,
        "Duplicate Status": is_dup,
        "Matched Hash": hash_match
    })

# Create DataFrame
df = pd.DataFrame(final_data)

# Save to CSV
csv_file_name = "AutoClaimAI_Output_Report.csv"
df.to_csv(csv_file_name, index=False)

# Download link
from google.colab import files
files.download(csv_file_name)